In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

In [ ]:
delimiter = "####"


system_message = f"""
You will be provided with movie viewer queries. \
The viewer query will be delimited with \
{delimiter} characters.
Output a python dictionary with following keys:
    'genre': <a list of of genres that must be found strictly in the allowed genre list below>,
    'title': <a list of movies that must be found strictly in the allowed movie list below>

Allowed genre list in our database:
Action
Drama
Comedy
Thriller
Adventure
Sci-Fi
Fantasy
Romance

Allowed movie list in out database:
The Secret Life of Dreams
Echoes of Eternity
Rhythm of Redemption
The Edge of Darkness

Only output the dictionary of objects, with nothing else.
"""


user_message_1 = f"""

I want to know about a movie its name was something like heros of eternity, \
also tell me about any sci-fi movie

"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
]

movies_and_genres_response_1 = get_completion_from_messages(messages)
print(movies_and_genres_response_1)

{\
    'genre': ['Sci-Fi'], \
    'title': ['Echoes of Eternity'] \
}

In [ ]:
movies_db = [
  {
    "title": "The Secret Life of Dreams",
    "genre": ["Fantasy", "Drama"],
    "rating": 8.2,
    "cast": ["Emma Thompson", "Tom Hanks", "Lily James"],
    "release_date": "2022-07-15"
  },
  {
    "title": "Echoes of Eternity",
    "genre": ["Mystery", "Thriller"],
    "rating": 7.6,
    "cast": ["Mark Ruffalo", "Emily Blunt", "Michael Fassbender"],
    "release_date": "2021-11-03"
  },
  {
    "title": "Rhythm of Redemption",
    "genre": ["Musical", "Romance"],
    "rating": 6.9,
    "cast": ["Anna Kendrick", "Chris Evans", "Zendaya"],
    "release_date": "2023-02-28"
  },
  {
    "title": "The Edge of Darkness",
    "genre": ["Action", "Sci-Fi"],
    "rating": 7.8,
    "cast": ["Scarlett Johansson", "Chris Pratt", "Robert Downey Jr."],
    "release_date": "2022-09-10"
  }
]


In [ ]:
def get_movie_by_name(name):
    return [movie for movie in movies_db if movie.get("title") == name]

def get_movie_by_genre(genre):
    return [movie for movie in movies_db if genre in movie.get("genre")]


In [ ]:
get_movie_by_name('The Secret Life of Dreams')

[{'title': 'The Secret Life of Dreams',\
  'genre': ['Fantasy', 'Drama'],\
  'rating': 8.2,\
  'cast': ['Emma Thompson', 'Tom Hanks', 'Lily James'],\
  'release_date': '2022-07-15'}]

In [ ]:
get_movie_by_genre('Romance')

[{'title': 'Rhythm of Redemption',\
  'genre': ['Musical', 'Romance'],\
  'rating': 6.9,\
  'cast': ['Anna Kendrick', 'Chris Evans', 'Zendaya'],\
  'release_date': '2023-02-28'}]

In [ ]:
def retrieve_info_of_required_movie(prompt_1_response):
    movies_and_genres_dict = eval(prompt_1_response)
    
    movies_by_title = [get_movie_by_name(title) for title in movies_and_genres_dict.get('title')]
    movies_by_genre = [get_movie_by_genre(genre) for genre in movies_and_genres_dict.get('genre')]
    
    movies_by_title = [item for sublist in movies_by_title for item in sublist]
    movies_by_genre = [item for sublist in movies_by_genre for item in sublist]
    
    merged_movies_list = []
    added_movies = []
    for movie_by_title in movies_by_title:
        for movie_by_genre in movies_by_genre:
            if movie_by_title.get('title') not in added_movies:
                merged_movies_list.append(movie_by_title)
                added_movies.append(movie_by_title.get('title'))
            if movie_by_genre.get('title') not in added_movies:
                merged_movies_list.append(movie_by_genre)
                added_movies.append(movie_by_genre.get('title'))
    return merged_movies_list

In [ ]:
retrieve_info_of_required_movie(movies_and_genres_response_1)

[{'title': 'Echoes of Eternity',\
  'genre': ['Mystery', 'Thriller'],\
  'rating': 7.6,\
  'cast': ['Mark Ruffalo', 'Emily Blunt', 'Michael Fassbender'],\
  'release_date': '2021-11-03'},\
 {'title': 'The Edge of Darkness',\
  'genre': ['Action', 'Sci-Fi'],\
  'rating': 7.8,\
  'cast': ['Scarlett Johansson', 'Chris Pratt', 'Robert Downey Jr.'],\
  'release_date': '2022-09-10'}]

In [ ]:
system_message = f"""
You are a movie information assistant. \
Respond with very concise answers based on the data retrieved from the database. \
Make sure to ask the user relevant follow up questions.
"""
user_message_1 = f"""

I want to know about a movie its name was something like heros of eternity, \
also tell me release dates of any sci-fi movie that is there.

"""
messages =  [  
{'role':'system',
 'content': system_message},   
{'role':'user',
 'content': user_message_1},  
{'role':'assistant',
 'content': f"""Relevant movie information:\n\
 {retrieve_info_of_required_movie(movies_and_genres_response_1)}"""},   
]
final_response = get_completion_from_messages(messages)
print(final_response)

I'm sorry, but there is no movie in the database with the title "Heros of Eternity". However, there is a movie called "Echoes of Eternity" which is a mystery thriller and has a rating of 7.6. Would you like to know more about this movie?

As for the release dates of sci-fi movies, there is one called "The Edge of Darkness" which is set to release on September 10th, 2022. Would you like to know more about this movie or any other sci-fi movies?